In [295]:
# Prerequisites
import pandas as pd
from pathlib import Path
from infowater.output.manager import Manager as OutMan
import arcpy

# Define Project information
project_name = "Purpletown"
scenario1 = "Base"
vdb_folder = "IWVDB1" # You can find this by viewing the Data Source tab on properties of your map layers
iwp_working_folder = "C:\\ProgramData\\Innovyze\\Temp\\IWPR10VDB"


In [296]:
# Create list of Junction ID's in Domain

junction_table = iwp_working_folder + "\\" + vdb_folder + "\\Map\\Map.gdb\\Junction"

# Create a list of junction ID's in Domain
cursor = arcpy.da.SearchCursor(junction_table, ["MOID"], "MOTYPE=1")
domain_junctions = []
for row in cursor:
    domain_junctions.append(row[0])

print(domain_junctions)

['J86', 'J88', 'J126', 'J128', 'J130', 'J154', 'J156', 'J420', 'J422', 'J424', 'J426', 'J428', 'J430', 'J432', 'J434', 'J436', 'J438', 'J440', 'J442', 'J444', 'J446', 'J448', 'J450', 'J452', 'J454', 'J456', 'J458', 'J460', 'J462', 'J464', 'J468', 'J472', 'J474', 'J476', 'J478', 'J480', 'J484', 'J486', 'J488', 'J490', 'J492', 'J494', 'J496', 'J498', 'J500', 'J502', 'J504', 'J506', 'J508', 'J510', 'J512', 'J514', 'J516', 'J518', 'J520', 'J522', 'J524', 'J526', 'J528', 'J530', 'J532', 'J534', 'J536', 'J538', 'J540', 'J542', 'J544', 'J546', 'J548', 'J550', 'J552', 'J554', 'J556', 'J558', 'J560', 'J562', 'J564', 'J566', 'J568', 'J570', 'J572', 'J574', 'J576', 'J578', 'J580', 'J582', 'J584', 'J586', 'J588', 'J590', 'J592', 'J594', 'J596', 'J598', 'J600', 'J852', 'J854', 'J856', 'J858', 'J860', 'J862', 'J864', 'J866', 'J868', 'J870', 'J872', 'J874', 'J876', 'J878', 'J880', 'J882', 'J884', 'J1334', 'J1336', 'J1338', 'J1340', 'J1342', 'J1344', 'J1348', 'J1350', 'J1352', 'J1354', 'J1358', 'J1360

In [297]:
# Create list of Junction ID's in Selection Set

# Input: Provide the name of the Selection Set
select_set = "TEST"

# Get table path. Note this applies to nodes only. Links requires the ALINK.DBF table.
anode_selection = vdb_path + "\\Select\\" +select_set+ "\\ANODE.DBF"

# Create a list of junction ID's in Selection Set
cursor = arcpy.da.SearchCursor(anode_selection, ["ID"])
sel_set_junctions = []
for row in cursor:
    sel_set_junctions.append(row[0])

print(sel_set_junctions)

['J86', 'J88', 'J126', 'J128', 'J130', 'J154', 'J156', 'J420', 'J422', 'J424', 'J426', 'J428', 'J430', 'J432', 'J434', 'J436', 'J438', 'J440', 'J442', 'J444', 'J446', 'J448', 'J450', 'J452', 'J454', 'J456', 'J458', 'J460', 'J462', 'J464', 'J468', 'J472', 'J474', 'J476', 'J478', 'J480', 'J484', 'J486', 'J488', 'J490', 'J492', 'J494', 'J496', 'J498', 'J500', 'J502', 'J504', 'J506', 'J508', 'J510', 'J512', 'J514', 'J516', 'J518', 'J520', 'J522', 'J524', 'J526', 'J528', 'J530', 'J532', 'J534', 'J536', 'J538', 'J540', 'J542', 'J544', 'J546', 'J548', 'J550', 'J552', 'J554', 'J556', 'J558', 'J560', 'J562', 'J564', 'J566', 'J568', 'J570', 'J572', 'J574', 'J576', 'J578', 'J580', 'J582', 'J584', 'J586', 'J588', 'J590', 'J592', 'J594', 'J596', 'J598', 'J600', 'J852', 'J854', 'J856', 'J858', 'J860', 'J862', 'J864', 'J866', 'J868', 'J870', 'J872', 'J874', 'J876', 'J878', 'J880', 'J882', 'J884', 'J1334', 'J1336', 'J1338', 'J1340', 'J1342', 'J1344', 'J1348', 'J1350', 'J1352', 'J1354', 'J1358', 'J1360

In [298]:
# Use above selection list to generate custom output table

# Get Outputs
aprx = arcpy.mp.ArcGISProject("CURRENT")
project_path = Path(aprx.filePath).parent
out1_path = str(project_path) + "/" + project_name + ".OUT/SCENARIO/" + scenario1 + "/HYDQUA.OUT"
out1 = OutMan(out1_path)

# Get list of indices based on selected junction ID's
junctions = out1.get_element_list("Junction")
j_indices = [i for i, moid in enumerate(junctions) if moid in domain_junctions]

# Get remaining results to build table
p1min = out1.get_range_data("Junction", "Pressure", "min")
p1avg = out1.get_range_data("Junction", "Pressure", "avg")
dmd1avg = out1.get_range_data("Junction", "Demand", "avg")

# Generate table
df = pd.DataFrame(
{
    "Junction": [junctions[i] for i in j_indices],
    "Base Avg Demand (gpm)": [dmd1avg[i]*448.8 for i in j_indices],
    "Base Avg Pressure (psi)": [p1avg[i]*0.433 for i in j_indices],
    "Base min Pressure (psi)": [p1min[i]*0.433 for i in j_indices],
})

df.sort_values(by='Base Avg Demand (gpm)',ascending=False)

,Junction,Base Avg Demand (gpm),Base Avg Pressure (psi),Base min Pressure (psi)
19,J1384,3.971385,52.377860,52.299718
11,J1348,1.717386,116.677484,113.479630
5,J1334,1.717377,107.847452,104.448386
18,J1364,1.707336,109.878269,107.710038
17,J1362,1.657122,108.960225,106.791862
...,...,...,...,...
118,J866,0.140677,96.607555,94.506604
96,J574,0.120587,99.436875,96.024509
97,J576,0.120585,98.220146,94.807780
1,8484,0.104585,73.669767,73.549631
